In [1]:
import os
import torch
import logging
logging.basicConfig(level=logging.ERROR)
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 4090


In [2]:
from transformers import AutoTokenizer, RobertaModel

# Load the RoBerta tokenizer.
print('Loading Roberta tokenizer...')
# tokenizer = AutoTokenizer.from_pretrained('witiko/mathberta', do_lower_case=True)
tokenizer = AutoTokenizer.from_pretrained('roberta-large-mnli', do_lower_case=True)

Loading Roberta tokenizer...


/home/jan/anaconda3/envs/pytorch/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
import json
# Changed from NumTemp-E9C0/output/bm25_top_100_train
with open("NumTemp-E9C0/output/bm25_top_100_train") as f:
    train_data = json.load(f)
with open('NumTemp-E9C0/output/bm25_top_100_val') as f:
    val_data = json.load(f)
with open('NumTemp-E9C0/output/bm25_top_100_test') as f:
    test_data = json.load(f)
len(train_data), len(val_data), len(test_data)

(9935, 3084, 2495)

In [5]:
train_data[-1]

{'country_of_origin': 'usa',
 'label': 'Conflicting',
 'url': 'https://www.politifact.com/factchecks/2016/may/03/alan-grayson/grayson-right-about-lack-paid-vacation-us-misses-d/',
 'lang': 'en',
 'claim': '"There\'s only three countries in the entire world wherethere\'s no paid vacation by law, we happen to be one of them."',
 'doc': 'American workers aren’t getting the compensation they deserve, in either time or money, U.S. Rep. Alan Grayson said at a recent Orlando debate. At a Senate campaign debate with U.S. Rep. David Jolly, R-Indian Shores, on April 25, 2016, Grayson said he supported a higher minimum wage. Providing more pay won’t hurt the businesses, he said, because other nations have higher wages with no ill effects. Furthermore, many countries also guarantee paid time off and their economies are fine, Grayson said. But not the United States. "There\'s only three countries in the entire world wherethere\'s no paid vacation by law, we happen to be one of them. The other two a

In [6]:
val_data[-1]

{'crawled_date': '2022-10-07T00:00:06',
 'country_of_origin': 'usa',
 'label': 'False',
 'url': 'https://www.snopes.com/fact-check/never-forget-january-6th/',
 'lang': 'en',
 'claim': 'Social media posts with pictures of gas prices and captions urging people to “Never forget January 6th” accurately documented the cost of fuel on the day of the 2021 U.S. Capitol riot.',
 'doc': 'The gas prices pictured in two different shared pictures did not reflect the national average price for regular unleaded gasoline on Jan. 6, 2021. Further, reverse image searches found that both photographs were taken while former U.S. President Barack Obama was in his second term in the White House. On June 12, 2022, the Cullman Daily Facebook page, which is managed by a self-described “conservative and Christian news organization,” posted a picture of an Exxon gas station sign showing the price for regular unleaded fuel at $1.73 per gallon, with the caption, “We will never forget January 6th, 2021.” At the tim

In [7]:
test_data[-1]

{'crawled_date': '2016-10-11',
 'country_of_origin': 'usa',
 'label': 'True',
 'url': 'https://www.politifact.com/factchecks/2016/oct/11/hillary-clinton/clinton-right-trump-tax-plan-gives-more-rich-bush-/',
 'lang': 'en',
 'claim': 'Says Donald Trump’s tax plan gives the wealthy and corporations "more than the Bush tax cuts by at least a factor of two."',
 'doc': 'With a backdrop of leaked lewd comments and Bill Clinton’s alleged sexual misconduct, you may not necessarily remember what Donald Trump and Hillary Clinton said about tax policy in the deeply personal and nasty Oct. 9 presidential debate. When Trump touted his proposal to deliver tax cuts for all, Clinton pointed out that his tax plan would help millionaires and billionaires like himself the most. "His plan will give the wealthy and corporations the biggest tax cuts they’ve ever had, more than the Bush tax cuts by at least a factor of two," Clinton said. "Donald always takes care of Donald and people like Donald, and this wo

In [8]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
k = 100

In [9]:
def get_features(data):
    features = []
    evidences = []
    
    for index, fact in enumerate(data):
        claim = fact["claim"]
        evidences = fact['top_n'][:k]
        evidences = ''.join(f'[Evidence]:{e}' for e in evidences)
        
        feature = "[Claim]:" + claim + evidences
        features.append(feature)
    return features

In [10]:
train_features = get_features(train_data)
val_features = get_features(val_data)
test_features = get_features(test_data)
len(train_features), len(val_features), len(test_data)

(9935, 3084, 2495)

In [11]:
train_features[-1]

'[Claim]:"There\'s only three countries in the entire world wherethere\'s no paid vacation by law, we happen to be one of them."[Evidence]:18 ago 2023  ... days of paid vacation in all eu countries. several countries offer even ... while swedes are guaranteed 25 paid vacation days by law, her ...[Evidence]:jun 22, 2023  if the plan becomes law, the united states will no longer be one of six countries in the world  and the only rich country  without any form ...[Evidence]:22 juin 2023  if the plan becomes law, the united states will no longer be one of six countries in the world  and the only rich country  without any form of ...[Evidence]:de r ray  2013  cit 105 fois  vacation and holiday laws. figure 1 summarizes the legal right to paid vacation for 21 of the richest countries in the world (see also table 1).24 pages[Evidence]:24 dc. 2019  for the next three decades, the two weeks of paid vacation time ... without any benefits, sought to pass a paid vacation law in the state.[Evidence

In [12]:
val_features[-1]

'[Claim]:Social media posts with pictures of gas prices and captions urging people to “Never forget January 6th” accurately documented the cost of fuel on the day of the 2021 U.S. Capitol riot.[Evidence]:2022-06-17  ... cost of fuel on the day of the 2021 u.s. capitol riot. conte. ... average price for regular unleaded gasoline on jan. 6, 2021. further, reverse ...[Evidence]:click on your local gas station on the fuel prices map to check out reviews and other information about each location. the average cost of us gas prices might[Evidence]:we try to make sure people have the cheapest price of fuel on the water $500 speedway fuel cards for $1.95 is a scam  yes, u.s. gas prices[Evidence]:6 oct. 2021  if you\'re using social media pictures as evidence, you\'ll need to verify the date they were taken. learn how to accurately date social media ...[Evidence]:6 ott 2021  if you\'re using social media pictures as evidence, you\'ll need to verify the date they were taken. learn how to accurate

In [12]:
test_features[-1]

'[Claim]:Says Donald Trump’s tax plan gives the wealthy and corporations "more than the Bush tax cuts by at least a factor of two."[Evidence]:his plan will give the wealthy and corporations the biggest tax cuts they\'ve ever had, more than the bush tax cuts by at least a factor of two.[Evidence]:hillary clinton reminded us: donald trump\'s tax plan "will give the wealthy and corporations the biggest tax cuts they\'ve ever had, more than the bush tax cuts by at least a factor of twothe way that he talks about his tax cuts would end up raising taxes on millions of middle-class families."[Evidence]:more than the bush tax cuts by at least a factor of two. donald to give the biggest tax breaks ever to the wealthy and to corporations.[Evidence]:mar 27, 2023  like the rest of the budget\'s investments, these tax cuts are more than paid for by ensuring wealthy people and large corporations pay their ...[Evidence]:this motion to concur on a bill that gives more tax cuts for the wealthy and corp

In [13]:
train_labels = [fact["label"] for fact in train_data]
val_labels = [fact["label"] for fact in val_data]
test_labels = [fact["label"] for fact in test_data]

train_labels_final = LE.fit_transform(train_labels)
val_labels_final = LE.transform(val_labels)
test_labels_final = LE.transform(test_labels)

train_labels_final[:20], val_labels_final[:20], test_labels_final[:20]

(array([1, 0, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 0, 0, 1, 2]),
 array([1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([0, 1, 1, 2, 1, 1, 0, 2, 0, 1, 0, 0, 2, 1, 1, 0, 0, 2, 0, 1]))

In [14]:
input_ids = []
attention_masks = []

for sent in train_features:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])
# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)


# Print sentence 0, now as a list of IDs.
print('Original: ', train_features[0])
print('Token IDs:', input_ids[0])

/home/jan/anaconda3/envs/pytorch/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  [Claim]:Says his political opponents "brought 100,000 protesters into our state."[Evidence]:jul 30, 2015  ... says that isn't a lie: says his political opponents "brought 100,000 protesters into our state." http://www.politifact.com/wisconsin ...[Evidence]:in a national tv interview, gov. scott walker said his political opponents "brought 100000 protesters into our state" during the 2011 demonstrations in ...[Evidence]:19 janv. 2021  a total loser! a loser who sadly plays right into our opponents hands! never a fan,.[Evidence]:31 ago 2023  ... up his political enemies if he is president again. in an ... his business affairs, trump said democrats and other opponents were sick people[Evidence]:jan 28, 2016  a loser who sadly plays right into our opponents hands! never a fan  a total lightweight  opposes me and some of our great republican ...[Evidence]:percent of the vote to clinch the gop nomination. polls had showed mccloskey was running far behind his opponents headed into 

In [15]:
val_input_ids = []
val_attention_masks = []

for sent in val_features:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    val_input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    val_attention_masks.append(encoded_dict['attention_mask'])
# Convert the lists into tensors.
val_input_ids = torch.cat(val_input_ids, dim=0)
val_attention_masks = torch.cat(val_attention_masks, dim=0)


# Print sentence 0, now as a list of IDs.
print('Original: ', val_features[0])
print('Token IDs:', val_attention_masks[0])

Original:  [Claim]:Amit Shah said Narendra Modi sleeps for 24 hours for the welfare of the poor.[Evidence]:2022-03-21  'pm modi sleeps for only two hours and works for 22 hours every day,' bjp chief chandrakant patil said.[Evidence]:2017-02-11  obama sleeps nearly five hours a night. narendra modi. pti. 2/2. narendra modi. prime minister narendra modi is said to work 20 hours a day.[Evidence]:mar 23, 2022  he sleeps for 3.5 to 4 hours every day. he sleeps late at around 12 o clock and gets up early around 4 o clock. yet, he is so fresh and energetic throughout the ...[Evidence]:mar 21, 2022  pm modi sleeps for only two hours and works for 22 hours every day. he is experimenting now so that he need not have to sleep," patil ...[Evidence]:sep 26, 2022  brett favre's most memorable stat may be $8 million meant for the poor. his role in the misappropriation of welfare money has infuriated ...[Evidence]:prime minister narendra modi sleeps for only two hours every day and is doing an experim

In [16]:
test_input_ids = []
test_attention_masks = []

for sent in test_features:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    test_input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    test_attention_masks.append(encoded_dict['attention_mask'])
# Convert the lists into tensors.
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)


# Print sentence 0, now as a list of IDs.
print('Original: ', test_features[0])
print('Token IDs:', test_attention_masks[0])

Original:  [Claim]:About 125 women took Mifepristone to terminate their pregnancies during a protest demanding the legalization of abortion in South Korea.[Evidence]:today, more than half of women who terminate their pregnancies rely on that drug. when mifepristone is used as fda directs, serious adverse[Evidence]:16 gen 2013  indeed, since the high court's january 1973 decision in roe v. wade, which granted women the constitutional right to terminate their pregnancies ...[Evidence]:30 ott 2019  [who] kneeled in protest of abortion during the national anthem in ... tim tebow was not protesting abortion when he took a knee at nfl games.[Evidence]:2015-11-17  according to a new study, texas' onerous abortion restrictions have forced women with unwanted pregnancies to take matters into their own ...[Evidence]:6 ott 2015  the regulation of abortion may vary widely from country to country, but nearly all nations  96%  allow women to terminate their ...[Evidence]:20 lug 2021  women hold lett

In [17]:
train_labels_final = torch.tensor(train_labels_final)
val_labels_final = torch.tensor(val_labels_final)
test_labels_final = torch.tensor(test_labels_final)

In [18]:
val_labels_final.shape, len(val_input_ids)

test_labels_final.shape, len(test_input_ids)

(torch.Size([2495]), 2495)

In [19]:
num_classes = len(list(set(train_labels)))
list(set(train_labels)), num_classes

(['True', 'Conflicting', 'False'], 3)

In [20]:
from torch.utils.data import TensorDataset, random_split
# train_poincare_tensor = torch.tensor(poincare_embeddings_final,dtype=torch.float)
# difficulty_tensor = torch.tensor(difficulty_level_vectors,dtype=torch.float)
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, train_labels_final)
val_dataset = TensorDataset(val_input_ids, val_attention_masks,val_labels_final)
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels_final)

In [21]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 16
train_dataloader = DataLoader(
            dataset,  # The training samples.
            sampler = RandomSampler(dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset),
            batch_size = batch_size
        )

In [22]:
from torch import nn
from transformers import AutoModel
class MultiClassClassifier(nn.Module):
    def __init__(self, roberta_model_path, labels_count, hidden_dim=768, mlp_dim=500, extras_dim=100, dropout=0.1, freeze_roberta=False):
        super().__init__()

        self.roberta = AutoModel.from_pretrained(roberta_model_path,output_hidden_states=True,output_attentions=True)
        self.dropout = nn.Dropout(dropout)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim, mlp_dim),
            nn.ReLU(),
            # nn.Linear(mlp_dim, mlp_dim),
            # nn.ReLU(),
            # nn.Linear(mlp_dim, mlp_dim),
            # nn.ReLU(),
            nn.Linear(mlp_dim, labels_count)
        )
        # self.softmax = nn.LogSoftmax(dim=1)
        if freeze_roberta:
            print("Freezing layers")
            for param in self.roberta.parameters():
                param.requires_grad = False

    def forward(self, tokens, masks):
        output = self.roberta(tokens, attention_mask=masks)
        dropout_output = self.dropout(output["pooler_output"])
        # concat_output = torch.cat((dropout_output, topic_emb), dim=1)
        # concat_output = self.dropout(concat_output)
        mlp_output = self.mlp(dropout_output)
        # proba = self.sigmoid(mlp_output)
        # proba = self.softmax(mlp_output)

        return mlp_output

In [31]:
from transformers import AdamW

# Loads RobertaModel, the pretrained RoBerta model with a single
model = MultiClassClassifier('uf-aice-lab/math-roberta',num_classes, 1024,768,140,dropout=0.3,freeze_roberta=False)

# model.load_state_dict(torch.load("model_bert_difficulty_prediction/model_weights"))

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of RobertaModel were not initialized from the model checkpoint at uf-aice-lab/math-roberta and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MultiClassClassifier(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
             

In [32]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [33]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [34]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [35]:
for param in model.roberta.encoder.layer[0:5].parameters():
    param.requires_grad=False

In [36]:
import random
import numpy as np
from transformers import get_linear_schedule_with_warmup

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42
epochs = 20

# To not get RuntimeError: CUDA error: device-side assert triggered (hopefully)
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)
loss_func = nn.CrossEntropyLoss()
# Total number of training steps is [number of batches] x [number of epochs].
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

# We'll store a number of quantities such as training and validation loss,
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
early_stopping = EarlyStopping(patience=2, verbose=True)
# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_accuracy = 0
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questimport gensim.downloader as api
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        # b_poincare = batch[2].to(device)
        # b_difficulty = batch[3].to(device)
        b_labels = batch[2].to(device)
        # skill_labels = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        probas = model(b_input_ids,b_input_mask)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        loss = loss_func(probas, b_labels)
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        # scheduler.step()
        logits = probas.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_train_accuracy += flat_accuracy(logits, label_ids)
    avg_train_accuracy = total_train_accuracy / len(train_dataloader)
    print(" Train Accuracy: {0:.2f}".format(avg_train_accuracy))

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)



    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)

        b_input_mask = batch[1].to(device)
        # b_poincare = batch[2].to(device)
        # b_difficulty = batch[3].to(device)
        b_labels = batch[2].to(device)
        # skill_labels = batch[3].to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():

            # Forward pass, calculate logit predictions.

          logits = model(b_input_ids,b_input_mask)

        # Accumulate the validation loss.
        loss = loss_func(logits, b_labels)
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)


    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    early_stopping(avg_val_loss, model)
    if early_stopping.early_stop:
      print("Early stopping")
      break
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
    output_dir = 'model_bart_large_oracle/'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print("Saving model to %s" % output_dir)
    tokenizer.save_pretrained(output_dir)
    torch.save(model.state_dict(), os.path.join(output_dir, 'model_weights'))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 20 ========
Training...
  Batch    40  of    621.    Elapsed: 0:00:11.
  Batch    80  of    621.    Elapsed: 0:00:22.
  Batch   120  of    621.    Elapsed: 0:00:34.
  Batch   160  of    621.    Elapsed: 0:00:45.
  Batch   200  of    621.    Elapsed: 0:00:56.
  Batch   240  of    621.    Elapsed: 0:01:07.
  Batch   280  of    621.    Elapsed: 0:01:18.
  Batch   320  of    621.    Elapsed: 0:01:29.
  Batch   360  of    621.    Elapsed: 0:01:40.
  Batch   400  of    621.    Elapsed: 0:01:51.
  Batch   440  of    621.    Elapsed: 0:02:02.
  Batch   480  of    621.    Elapsed: 0:02:13.
  Batch   520  of    621.    Elapsed: 0:02:25.
  Batch   560  of    621.    Elapsed: 0:02:36.
  Batch   600  of    621.    Elapsed: 0:02:47.
 Train Accuracy: 0.62

  Average training loss: 0.85
  Training epcoh took: 0:02:53

Running Validation...
  Accuracy: 0.65
Validation loss decreased (inf --> 0.770641).  Saving model ...
  Validation Loss: 0.77
  Validation took: 0:00:20
Saving model

In [37]:
# Directory where the model and tokenizer are saved
output_dir = 'model_bart_large_oracle/'

# Load the saved model
model = MultiClassClassifier('uf-aice-lab/math-roberta',num_classes, 1024,768,140,dropout=0.3,freeze_roberta=False)
model.load_state_dict(torch.load(os.path.join(output_dir, 'model_weights')))
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at uf-aice-lab/math-roberta and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MultiClassClassifier(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
             

In [38]:
taxonomy_classes = [fact["taxonomy_label"] for fact in test_data]
list(set(taxonomy_classes)), len(set(taxonomy_classes))

(['temporal   ', 'comparison', 'statistical', 'interval', 'temporal'], 5)

In [39]:
from sklearn.metrics import accuracy_score, classification_report

# Assuming the categories are represented as integers: 0 - temporal, 1 - statistical, 2 - comparison, 3 - interval
category_names = ["temporal", "statistical", "comparison", "interval"]
all_labels = [0, 1, 2, 3]

# Set the model to evaluation mode
model.eval()

# Tracking variables
total_test_accuracy = 0
total_test_loss = 0
total_test_steps = 0

# Create containers for true labels and predictions for detailed classification report
true_labels = []
predictions = []

print("Running Test Evaluation...")

# Measure how long the test run takes.
t0 = time.time()

# Evaluate data for one epoch
for batch in test_dataloader:

    # Unpack this test batch from our dataloader.
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():
        # Forward pass, calculate logit predictions.
        logits = model(b_input_ids, b_input_mask)

    # Accumulate the test loss.
    loss = loss_func(logits, b_labels)
    total_test_loss += loss.item()

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store true labels and predictions for later analysis
    true_labels.extend(label_ids)
    predictions.extend(np.argmax(logits, axis=1))

    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_test_accuracy += flat_accuracy(logits, label_ids)

# Calculate the average loss over all of the batches.
avg_test_loss = total_test_loss / len(test_dataloader)

# Report the final accuracy for this test run.
avg_test_accuracy = total_test_accuracy / len(test_dataloader)
print("  Test Accuracy: {0:.2f}".format(avg_test_accuracy))

# Classification report for detailed category-wise performance
classification_report_str = classification_report(true_labels, predictions, target_names=category_names, labels=all_labels)
print(classification_report_str)

# Measure how long the test run took.
test_time = format_time(time.time() - t0)

print("  Test Loss: {0:.2f}".format(avg_test_loss))
print("  Test took: {:}".format(test_time))

print("")
print("Test Evaluation complete!")

# Save the classification report to a file if needed
output_dir = 'model_bart_large_oracle/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

with open(os.path.join(output_dir, "test_classification_report.txt"), "w") as writer:
    writer.write(classification_report_str)

# If you want to save the test results
with open(os.path.join(output_dir, "test_results.txt"), "w") as writer:
    writer.write("Test Loss: {0:.2f}\n".format(avg_test_loss))
    writer.write("Test Accuracy: {0:.2f}\n".format(avg_test_accuracy))
    writer.write("Test took: {:}\n".format(test_time))


Running Test Evaluation...
  Test Accuracy: 0.62
              precision    recall  f1-score   support

    temporal       0.42      0.42      0.42       598
 statistical       0.83      0.71      0.76      1423
  comparison       0.45      0.63      0.52       474
    interval       0.00      0.00      0.00         0

   micro avg       0.62      0.62      0.62      2495
   macro avg       0.42      0.44      0.43      2495
weighted avg       0.66      0.62      0.63      2495

  Test Loss: 0.98
  Test took: 0:00:16

Test Evaluation complete!


/home/jan/anaconda3/envs/pytorch/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jan/anaconda3/envs/pytorch/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jan/anaconda3/envs/pytorch/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

In [40]:
from collections import Counter

# Counter for the number of claims in each category
taxonomy_counter = Counter()

# Iterate through the test_dataloader to count the number of claims in each category
for batch in test_dataloader:
    b_labels = batch[2].to('cpu').numpy()  # Assuming the labels are in the third position of the batch
    taxonomy_counter.update(b_labels)

# Print the number of claims in each category
print("Number of claims in each category in the test data:")
for i, count in taxonomy_counter.items():
    print(f"{category_names[i]}: {count}")

Number of claims in each category in the test data:
temporal: 598
statistical: 1423
comparison: 474


In [41]:
# print(test_dataset[0])
# print(test_data[0])
# print(predictions)
print(true_labels)

[0, 1, 1, 2, 1, 1, 0, 2, 0, 1, 0, 0, 2, 1, 1, 0, 0, 2, 0, 1, 1, 0, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 2, 1, 0, 1, 1, 0, 0, 1, 1, 2, 1, 1, 0, 1, 1, 1, 2, 2, 0, 1, 2, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 0, 0, 1, 2, 0, 1, 1, 2, 1, 2, 1, 1, 1, 0, 2, 2, 1, 1, 0, 1, 1, 0, 2, 1, 0, 1, 1, 0, 0, 2, 0, 1, 1, 2, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 2, 0, 2, 1, 2, 0, 1, 1, 1, 1, 1, 1, 2, 2, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 2, 1, 2, 1, 1, 1, 0, 1, 2, 1, 0, 0, 1, 1, 0, 0, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 0, 1, 0, 1, 1, 0, 1, 2, 2, 0, 0, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 0, 2, 1, 0, 1, 0, 0, 2, 1, 1, 1, 1, 2, 1, 0, 1, 1, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 2, 0, 1, 1, 1, 2, 0, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 1, 

In [42]:
# Code for qualitative analysis
import random

# Set a random seed for reproducibility
seed_val = 42
random.seed(seed_val)

# True labels and predictions have been collected during the test evaluation
# true_labels and predictions are lists of integers representing the labels
label_to_string = lambda x: {0: "True", 1: "Conflicting", 2: "False"}.get(x, "Invalid label")

# Identify failed predictions
failed_indices = [i for i in range(len(true_labels)) if true_labels[i] != predictions[i]]

# Randomly select 5 failed predictions
if len(failed_indices) < 5:
    print("Not enough failed predictions to sample 5. Sampling all failed predictions.")
    sample_indices = failed_indices
else:
    sample_indices = random.sample(failed_indices, 5)  
    
# Collecting the selected claims, their IDs and labels
selected_claims = []
selected_claim_ids = []
selected_true_labels = []
selected_pred_labels = []

for idx in sample_indices:
    selected_claims.append(test_dataset[idx][0])  # Assuming 'input_ids' is the first element
    # selected_claim_ids.append(test_dataset[idx][3])  # Assuming 'id' is the fourth element
    selected_true_labels.append(true_labels[idx])
    selected_pred_labels.append(predictions[idx])
    

# Print the selected claims, their IDs, true labels and predicted labels
print("Randomly selected failed claims:")
for i in range(len(selected_claims)):
    print(f"ID: {sample_indices[i]}")
    print(f"True Label: {test_data[i]['label']}")
    print(f"Predicted Label: {label_to_string(selected_pred_labels[i])}")
    print(f"Taxonomy label: {test_data[i]['taxonomy_label']}")
    print(f"Claim: {tokenizer.decode(selected_claims[i], skip_special_tokens=True)}")
    print("----------")

# Print the seed used
print(f"Random seed used for selection: {seed_val}")

Randomly selected failed claims:
ID: 1747
True Label: Conflicting
Predicted Label: Conflicting
Taxonomy label: statistical
Claim: [Claim]:Photographs accurately show the two singers Blake Shelton and Gwen Stefani getting married in July 2021.[Evidence]:06:21:53 am ist, 26 january 2021. two lakh tractors are expected to participate in the parade in a show of strength and support. 06:21:32 am[Evidence]:to illustrate this point, we accurately describe the propagation of covid-19 in spain using one such model and show that predictions for its subsequent...[Evidence]:2022-07-16  despite being married for decades, little is known about van der donk and d'onofrio's relationship. they married in 1997 and shared two sons...[Evidence]:latest fcdo travel advice for france including on entry requirements, safety and security and local laws and customs. getting married abroad. requirements,...[Evidence]:24 fvr. 2017  the most vulgar song in hindi by two bollywood singers is choot - volume i 'yo yo 